In [1]:
import pandas as pd
import json

def process_reconciliation_data(input_file, output_file):
    """
    Reads transaction reconciliation data, filters 'Not Found-SysB' cases,
    extracts 'amount' and 'fee' from JSON fields, and saves the cleaned file.
    """
    # Load data
    df = pd.read_csv(input_file, dtype=str)

    # Fill missing values with empty string
    df.fillna("", inplace=True)

    # Filter 'Not Found-SysB' cases
    df = df[df["recon_status"] == "Not Found"]

    # Extract 'amount' and 'fee' from JSON in 'recon_sub_status'
    def extract_json_data(json_str):
        try:
            data = json.loads(json_str.replace("'", "\""))  # Fix single quotes if any
            return data.get("amount", ""), data.get("fee", "")
        except:
            return "", ""

    df[["sys_b_amount", "sys_b_fee"]] = df["recon_sub_status"].apply(lambda x: pd.Series(extract_json_data(x)))

    # Select relevant columns
    cleaned_df = df[["txn_ref_id", "sys_a_date", "sys_a_amount_attribute_1", "sys_a_amount_attribute_2", "sys_b_amount", "sys_b_fee", "currency_type"]]

    # Save to CSV
    cleaned_df.to_csv(output_file, index=False)
    print(f"Processed file saved as: {output_file}")

# Example usage
process_reconciliation_data("recon_data_raw.csv", "recon_data_processed.csv")

Processed file saved as: recon_data_processed.csv


In [2]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

import shutil

def upload_file(file_path, upload_folder="processed_files"):
    """
    Moves the file to a specified folder.
    If a file with the same name exists, it appends a timestamp to avoid overwriting.
    """
    if not os.path.exists(upload_folder):
        os.makedirs(upload_folder)

    filename = os.path.basename(file_path)
    destination_path = os.path.join(upload_folder, filename)

    # Ensure uniqueness
    if os.path.exists(destination_path):
        timestamp = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")
        destination_path = os.path.join(upload_folder, f"{timestamp}_{filename}")

    shutil.move(file_path, destination_path)
    print(f"File uploaded to: {destination_path}")

def send_email(recipient_email, subject, body, attachment_path):
    """
    Sends an email with the processed CSV as an attachment.
    """
    sender_email = os.getenv("SMTP_EMAIL")
    sender_password = os.getenv("SMTP_PASSWORD")
    smtp_server = os.getenv("SMTP_SERVER", "smtp.example.com")
    smtp_port = int(os.getenv("SMTP_PORT", "587"))

    if not sender_email or not sender_password:
        raise ValueError("Missing SMTP credentials! Set them as environment variables.")

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = recipient_email
    msg["Subject"] = subject

    with open(attachment_path, "rb") as attachment:
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header("Content-Disposition", f"attachment; filename={os.path.basename(attachment_path)}")
        msg.attach(part)

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient_email, msg.as_string())

    print(f"Email sent to {recipient_email}")

# Example usage
upload_file("recon_data_processed.csv")
# send_email("finance@zenstatement.com", "Processed Transactions", "Please find attached.", "processed_files/categorized_transactions.csv")


File uploaded to: processed_files\20250210205627_recon_data_processed.csv


In [ ]:
import chardet
import pandas as pd
import ollama
import os
import argparse

# Define output folders
RESOLVED_FOLDER = "resolved_cases"
UNRESOLVED_FOLDER = "unresolved_cases"
PATTERN_DATABASE = "resolution_patterns.csv"

# Create folders if they don't exist
os.makedirs(RESOLVED_FOLDER, exist_ok=True)
os.makedirs(UNRESOLVED_FOLDER, exist_ok=True)

# Load or initialize the pattern database
if os.path.exists(PATTERN_DATABASE):
    try:
        pattern_df = pd.read_csv(PATTERN_DATABASE)
        known_patterns = set(pattern_df["Pattern"].dropna().tolist())
    except pd.errors.EmptyDataError:
        known_patterns = set()
else:
    known_patterns = set()

def query_llm(prompt):
    """
    Queries the Ollama LLM and handles any API errors.
    """
    try:
        response = ollama.chat(
            model="llama3.1:8b",
            messages=[{"role": "user", "content": prompt}]
        )
        return response["message"]["content"].strip()
    except Exception as e:
        print(f"LLM Query Failed: {e}")
        return "Error: Could not generate response."

def classify_resolution_status(text):
    """
    Classifies the resolution status as 'Resolved' or 'Unresolved'.
    """
    return query_llm(f"Classify the following financial issue as 'Resolved' or 'Unresolved': {text}")

def generate_summary(text):
    """
    Generates a summary for the given financial issue.
    """
    return query_llm(f"Summarize this financial issue: {text}")

def suggest_next_steps(text):
    """
    Suggests next steps for resolving the given financial issue.
    """
    return query_llm(f"Suggest next steps for resolving: {text}")

def identify_resolution_pattern(text):
    """
    Identifies the resolution pattern from the case comments.
    """
    return query_llm(f"Identify resolution pattern from this case: {text}")

def process_resolutions(resolution_file):
    """
    Processes resolution comments from the given file, classifies them as resolved or unresolved,
    and saves the categorized data into respective folders.
    """

    # Detect file encoding
    with open(resolution_file, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']

    # Read CSV with detected encoding
    df = pd.read_csv(resolution_file, encoding=encoding)

    # Initialize lists to store resolved and unresolved case data
    unresolved_data = []
    resolved_data = []
    new_patterns = []

    for _, row in df.iterrows():
        order_id = row["Transaction ID"]
        amount = row["amount"]
        comments = row["Comments"]

        # Step 1: Use LLM to classify as Resolved or Unresolved
        resolution_status = classify_resolution_status(comments)

        if "unresolved" in resolution_status.lower():
            # Step 2: Handle Unresolved Cases
            summary = generate_summary(comments)
            next_steps = suggest_next_steps(comments)
            print(f"[{order_id}] Unresolved - Summary: {summary}, Next Steps: {next_steps}")

            # Append unresolved case data
            unresolved_data.append([order_id, amount, comments, summary, next_steps])

            # Save unresolved cases after each iteration (append mode)
            unresolved_df = pd.DataFrame(unresolved_data, columns=["Transaction ID", "Amount", "Comments", "Summary", "Next Steps"])
            unresolved_df.to_csv(f"{UNRESOLVED_FOLDER}/unresolved_cases.csv", mode='a', header=not os.path.exists(f"{UNRESOLVED_FOLDER}/unresolved_cases.csv"), index=False)

        else:
            # Step 3: Handle Resolved Cases
            pattern = identify_resolution_pattern(comments)
            
            if pattern in known_patterns:
                print(f"[{order_id}] Resolved - Auto-closed due to recognized pattern: {pattern}")
                auto_closed = "Yes"
            else:
                print(f"[{order_id}] Resolved - Identified New Pattern: {pattern}")
                auto_closed = "No"
                new_patterns.append(pattern)

            # Append resolved case data
            resolved_data.append([order_id, amount, comments, pattern, auto_closed])

            # Save resolved cases after each iteration (append mode)
            resolved_df = pd.DataFrame(resolved_data, columns=["Transaction ID", "Amount", "Comments", "Pattern", "Auto Closed"])
            resolved_df.to_csv(f"{RESOLVED_FOLDER}/resolved_cases.csv", mode='a', header=not os.path.exists(f"{RESOLVED_FOLDER}/resolved_cases.csv"), index=False)

    # Step 4: Update pattern database
    if new_patterns:
        new_patterns_df = pd.DataFrame({"Pattern": new_patterns})
        new_patterns_df.to_csv(PATTERN_DATABASE, mode='a', header=False, index=False)

# Example usage
process_resolutions("recon_data_reply.csv")


[44516715] Unresolved - Summary: The company's financial records show that a specific payment is missing from its transaction history, and further review is needed to determine what happened to it., Next Steps: A common issue! Here are some suggested next steps to resolve the problem:

**Initial Steps**

1. **Verify the report and payment details**: Double-check that the payment ID, date, amount, and financial report are accurate.
2. **Check for errors in payment processing**: Ensure that the payment was processed correctly and that there were no issues with the bank transfer or payment gateway.

**Additional Investigation Steps**

1. **Review accounting records**: Check if the payment has been recorded in the general ledger or other relevant accounts.
2. **Contact the payment recipient**: Reach out to the person who made the payment (if applicable) to confirm receipt and any potential errors.
3. **Check for discrepancies in financial reports**: Investigate if there are any issues with